<a href="https://colab.research.google.com/github/R12942159/NTU_ML/blob/Hw2/CNN_pretrained.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import random
import glob
import csv
import torch
import torch.nn as nn
import numpy as np
import pandas as pd

from torch.optim import Adam
from torchvision import transforms as tr
from torch.utils.data import DataLoader, Dataset
from PIL import Image

In [ ]:
from google.colab import files
files.upload()
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
!kaggle competitions download -c ml2023-fall-hw2
!unzip 'ml2023-fall-hw2.zip'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using: {device}")

Using: cuda


In [ ]:
import pandas as pd

img_id = pd.read_csv('/content/2023HW2_7000/train.csv')['id'].values.tolist()
img_labels = pd.read_csv('/content/2023HW2_7000/train.csv')['label'].values.tolist()

paths = '/content/2023HW2_7000/'
train_paths = [f'{paths}train/{i}.jpg' for i in img_id]
test_paths = [f'{paths}test/{i}.jpg' for i in range(0, 7000)]

In [ ]:
from PIL import Image


IMG_SIZE = Image.open(train_paths[10]).convert('RGB')
IMG_SIZE = int(np.array(IMG_SIZE).shape[1])

In [ ]:
import torch
import re


class Hw2_train_ds(torch.utils.data.Dataset):
    def __init__(self, img_path, img_label, transform, augmentation=True) -> None:
        self.img_path = img_path
        self.img_label = img_label
        self.transform = transform
        self.augmentation = augmentation

        hflip = tr.RandomHorizontalFlip(p=0.5)
        vflip = tr.RandomVerticalFlip(p=0.5)
        rotate = tr.RandomRotation(degrees=15)
        color = tr.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1)

        def origine(x): return x
        if augmentation:
            self.augmentation = [hflip, vflip, rotate, color]
        else:
            self.augmentation = [origine]

        assert len(self.img_path) == len(self.img_label)

    def __len__(self):
        return len(self.img_path)

    def __getitem__(self, idx):
        # read img
        path = self.img_path[idx]
        img = Image.open(path).convert('RGB') # img = (64,64,3)

        # transform/normalize img
        img = self.transform(img)

        # augmentation img
        augment = random.choice(self.augmentation)
        img = augment(img)

        # get label
        label = self.img_label[idx]
        label = torch.tensor(label)

        return img, label

class Hw2_test_ds(torch.utils.data.Dataset):
    def __init__(self, img_path, transform):
        self.img_path = img_path
        self.transform = transform

    def __len__(self):
        return len(self.img_path)

    def __getitem__(self, idx):
        path = self.img_path[idx]
        img = Image.open(path).convert('RGB')
        img = self.transform(img)

        return img, re.split(r'[./]', path)[-2]

In [ ]:
from torchvision.models import resnet50, ResNet50_Weights


rn50_transform = ResNet50_Weights.DEFAULT.transforms()

img_ds = Hw2_train_ds(train_paths,
                      img_labels,
                      transform = rn50_transform,
                      augmentation = True,
                      )

test_ds = Hw2_test_ds(test_paths,
                     transform = rn50_transform,
                      )

In [ ]:
rn50_transform

ImageClassification(
    crop_size=[224]
    resize_size=[232]
    mean=[0.485, 0.456, 0.406]
    std=[0.229, 0.224, 0.225]
    interpolation=InterpolationMode.BILINEAR
)

In [ ]:
from torch.utils.data import DataLoader, random_split


# Randomly divided into a training and validation dataset.
dataset_size = len(img_ds)
val_size = int(0.12 * dataset_size)
train_ds, val_ds = random_split(img_ds, [dataset_size - val_size, val_size])

In [ ]:
# Build dataloders
import torch

BATCH_SIZE = 64

train_loader = torch.utils.data.DataLoader(train_ds, BATCH_SIZE, shuffle=True, num_workers=0)
val_loader = torch.utils.data.DataLoader(val_ds, BATCH_SIZE * 2, shuffle=False, num_workers=0)
test_loader = torch.utils.data.DataLoader(test_ds, BATCH_SIZE, shuffle=False, num_workers=0)

#### Build Model

In [ ]:
model = resnet50(weights=ResNet50_Weights.DEFAULT)
print(model)

In [ ]:
# ResNet50
classifier = nn.Sequential(
    nn.Dropout(0.5),
    nn.Linear(2048, 7, bias=True),
    # nn.ReLU(),
    # nn.Dropout(0.3),
    # nn.Linear(128, 7)
)

model.fc = classifier

# Effeficient_b4
# clf = list(model.children())[-1]

# weights = clf[1].weight
# bias = clf[1].bias

# re_clf = torch.nn.Linear(in_features=weights.shape[1], out_features=7, bias=True)

# re_clf.weight.data = weights
# re_clf.bias.data = bias

# clf[1] = re_clf

In [ ]:
from torchsummary import summary


resize_size = rn50_transform.resize_size[0]
summary(model.to(device), (3, resize_size, resize_size))

#### Define training and testing process

In [ ]:
from tqdm.auto import tqdm # (optional) progress bar


def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset) # number of samples
    num_batches = len(dataloader) # batches per epoch

    model.train() # to training mode.
    epoch_loss, epoch_correct = 0, 0
    for batch_i, (x, y) in enumerate(tqdm(dataloader, leave=False)):
        x, y = x.to(device), y.to(device) # move data to GPU

        pred = model(x)
        loss = loss_fn(pred, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # write to logs
        epoch_loss += loss.item() # tensor -> python value
        # (N, Class)
        epoch_correct += (pred.argmax(dim=1) == y).sum().item()

    # return avg loss of epoch, acc of epoch
    return epoch_loss/num_batches, epoch_correct/size


def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset) # number of samples
    num_batches = len(dataloader) # batches per epoch

    model.eval() # model to test mode.
    epoch_loss, epoch_correct = 0, 0

    # No gradient for test data
    with torch.no_grad():
        for batch_i, (x, y) in enumerate(dataloader):
            x, y = x.to(device), y.to(device)

            pred = model(x)
            loss = loss_fn(pred, y)

            # write to logs
            epoch_loss += loss.item()
            epoch_correct += (pred.argmax(1) == y).sum().item()

    return epoch_loss/num_batches, epoch_correct/size

In [ ]:
EPOCHS = 100

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=3e-4, weight_decay=1e-6)

logs = {
    'train_loss': [], 'train_acc': [], 'val_loss': [], 'val_acc': []
}

# early stopping
patience = 15
counter = 0
best_loss = np.inf

for epoch in tqdm(range(EPOCHS)):
    train_loss, train_acc = train(train_loader, model, loss_fn, optimizer)
    val_loss, val_acc = test(val_loader, model, loss_fn)

    print(f'EPOCH: {(epoch+1):04d} \
    train_loss: {train_loss:.4f}, train_acc: {train_acc:.3f} \
    val_loss: {val_loss:.4f}, val_acc: {val_acc:.3f} ')

    logs['train_loss'].append(train_loss)
    logs['train_acc'].append(train_acc)
    logs['val_loss'].append(val_loss)
    logs['val_acc'].append(val_acc)


    torch.save(model.state_dict(), f'/content/drive/MyDrive/NTU_ML/Hw2/epoch{epoch+1}_model.pth')
    # chcek improvement
    if val_loss <  best_loss:
        counter = 0
        best_loss = val_loss
        torch.save(model.state_dict(), 'DL_hw2_best_model.pth')
        torch.save(model.state_dict(), '/content/drive/MyDrive/NTU_ML/Hw2/ResNet50_best_model.pth')
        # torch.save(optimizer.state_dict(), 'DL_hw2_best_optimizer.pth')
        print('-------------------- Model Save --------------------')
    else:
        counter += 1
    if counter >= patience:
        print('-------------------- Early Stopping --------------------')
        break

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/398 [00:00<?, ?it/s]

EPOCH: 0001     train_loss: 1.3482, train_acc: 0.478     val_loss: 1.1695, val_acc: 0.549 
-------------------- Model Save --------------------


  0%|          | 0/398 [00:00<?, ?it/s]

EPOCH: 0002     train_loss: 1.0866, train_acc: 0.592     val_loss: 1.0708, val_acc: 0.589 
-------------------- Model Save --------------------


...

EPOCH: 0019     train_loss: 0.2833, train_acc: 0.904     val_loss: 1.3816, val_acc: 0.636 


  0%|          | 0/398 [00:00<?, ?it/s]

EPOCH: 0020     train_loss: 0.2742, train_acc: 0.905     val_loss: 1.2925, val_acc: 0.628 


  0%|          | 0/398 [00:00<?, ?it/s]

KeyboardInterrupt: ignored

In [ ]:
def test(test_loader, model, file_name='predict.csv'):
    with torch.no_grad():
        predict_result = []
        predict_name = []
        for img, name in test_loader:
            img = img.to(device)
            pred = model(img)
            predict = torch.argmax(pred, dim=-1).tolist()
            predict_result += predict
            predict_name += name

    with open(file_name, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(['id', 'label'])
        for id, r in zip(predict_name, predict_result):
            writer.writerow([id, r])

In [ ]:
# del model
# model = resnet50(weights=ResNet50_Weights.DEFAULT)
model.load_state_dict(torch.load('/content/drive/MyDrive/NTU_ML/Hw2/epoch19_model.pth'))
model = model.to(device)
test(test_loader, model)